In [ ]:
"""
Copyright (c) Microsoft Corporation.
Licensed under the MIT license.

MIMIC-III Sepsis Cohort Extraction.

This file is sourced and modified from: https://github.com/matthieukomorowski/AI_Clinician
"""

import argparse
import os

import pandas as pd
import psycopg2 as pg

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# Set up environment variables
project_id = 'big-coil-431202-q6'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

dataset = 'mimiciii'

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')



Extraction of sub-tables
There are 43 tables in the Mimic III database.
26 unique tables; the other 17 are partitions of chartevents that are not to be queried directly
See: https://mit-lcp.github.io/mimic-schema-spy/
We create 15 sub-tables when extracting from the database

From each table we extract subject ID, admission ID, ICU stay ID
and relevant times to assist in joining these tables
All other specific information extracted will be documented before each section of the following code.


NOTE: The next three tables are built to help identify when a patient may be
considered to be septic, using the Sepsis 3 criteria

1. culture
These correspond to blood/urine/CSF/sputum cultures etc
There are 18 chartevent tables in the Mimic III database, one unsubscripted and
the others subscripted from 1 to 17. We use the unsubscripted one to create the
culture subtable. The remaining 17 are just partitions and should not be directly queried.
The labels corresponding to the 51 itemids in the query below are:


 Itemid | Label
-----------------------------------------------------
    938 | blood cultures
    941 | urine culture
    942 | BLOOD CULTURES
    2929 | sputum culture
    3333 | Blood Cultures
    4855 | Urine culture
    6035 | Urinalysis sent
    6043 | surface culture
    70006 | ANORECTAL/VAGINAL CULTURE
    70011 | BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)
    70012 | BLOOD CULTURE
    70013 | FLUID RECEIVED IN BLOOD CULTURE BOTTLES
    70014 | BLOOD CULTURE - NEONATE
    70016 | BLOOD CULTURE (POST-MORTEM)
    70024 | VIRAL CULTURE: R/O CYTOMEGALOVIRUS
    70037 | FOOT CULTURE
    70041 | VIRAL CULTURE:R/O HERPES SIMPLEX VIRUS
    70055 | POSTMORTEM CULTURE
    70057 | Rapid Respiratory Viral Screen & Culture
    70060 | Stem Cell - Blood Culture
    70063 | STERILITY CULTURE
    70075 | THROAT CULTURE
    70083 | VARICELLA-ZOSTER CULTURE
    80220 | AFB GROWN IN CULTURE; ADDITIONAL INFORMATION TO FOLLOW
    225401 | Blood Cultured
    225437 | CSF Culture
    225444 | Pan Culture
    225451 | Sputum Culture
    225454 | Urine Culture
    225722 | Arterial Line Tip Cultured
    225723 | CCO PAC Line Tip Cultured
    225724 | Cordis/Introducer Line Tip Cultured
    225725 | Dialysis Catheter Tip Cultured
    225726 | Tunneled (Hickman) Line Tip Cultured
    225727 | IABP Line Tip Cultured
    225728 | Midline Tip Cultured
    225729 | Multi Lumen Line Tip Cultured
    225730 | PA Catheter Line Tip Cultured
    225731 | Pheresis Catheter Line Tip Cultured
    225732 | PICC Line Tip Cultured
    225733 | Indwelling Port (PortaCath) Line Tip Cultured
    225734 | Presep Catheter Line Tip Cultured
    225735 | Trauma Line Tip Cultured
    225736 | Triple Introducer Line Tip Cultured
    225768 | Sheath Line Tip Cultured
    225814 | Stool Culture
    225816 | Wound Culture
    225817 | BAL Fluid Culture
    225818 | Pleural Fluid Culture
    226131 | ICP Line Tip Cultured
    227726 | AVA Line Tip Cultured



In [ ]:

culture = run_query("""
select subject_id, hadm_id, icustay_id, UNIX_SECONDS(CAST(charttime AS TIMESTAMP)) AS charttime, itemid
from `physionet-data.mimiciii_clinical.chartevents`
where itemid in (6035, 3333, 938, 941, 942, 4855, 6043, 2929, 225401, 225437, 225444, 225451, 225454, 225814,
  225816, 225817, 225818, 225722, 225723, 225724, 225725, 225726, 225727, 225728, 225729, 225730, 225731,
  225732, 225733, 227726, 70006, 70011, 70012, 70013, 70014, 70016, 70024, 70037, 70041, 225734, 225735,
  225736, 225768, 70055, 70057, 70060, 70063, 70075, 70083, 226131, 80220)
order by subject_id, hadm_id, charttime
""")

culture.to_csv("culture.csv",index=False)


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [ ]:
# 2. microbio (Microbiologyevents)
microbio = run_query("""
select subject_id, hadm_id, UNIX_SECONDS(CAST(charttime AS TIMESTAMP)) AS charttime, UNIX_SECONDS(CAST(chartdate as TIMESTAMP)) as chartdate
from `physionet-data.mimiciii_clinical.microbiologyevents`
""")

# Print the DataFrame to display the results
print(microbio)
microbio.to_csv('microbio.csv',index=False)

<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


        subject_id  hadm_id   charttime   chartdate
0            40822   102495  4214284980  4214246400
1            55801   165940  5974789920  5974732800
2            72908   176846  4988088000  4988044800
3            72978   198761  6923190540  6923145600
4            76600   197900  4212295200  4212259200
...            ...      ...         ...         ...
631721       90403   103315        <NA>  6648739200
631722       95929   160996        <NA>  6952953600
631723       98279   182617        <NA>  5015174400
631724       99231   168976        <NA>  5668617600
631725       47785   148784        <NA>  5067446400

[631726 rows x 4 columns]


In [ ]:

# 3. abx (Antibiotics administration)
# gsn/GSN: Generic Sequence Number. This number provides a representation of the drug in various coding systems.
# GSN is First DataBank's classification system. These are 6 digit codes for various drugs.
# ???  The codes here correspond to various antibiotics as sepsis onset is detected by administration of antibiotcs ???
abx = run_query("""
SELECT
    hadm_id,
    icustay_id,
    UNIX_SECONDS(CAST(startdate AS TIMESTAMP)) AS startdate_epoch,
    UNIX_SECONDS(CAST(enddate AS TIMESTAMP)) AS enddate_epoch
FROM
    `physionet-data.mimiciii_clinical.prescriptions`
WHERE
    gsn IN ('002542', '002543', '007371', '008873', '008877', '008879', '008880', '008935',
            '008941', '008942', '008943', '008944', '008983', '008984', '008990', '008991',
            '008992', '008995', '008996', '008998', '009043', '009046', '009065', '009066',
            '009136', '009137', '009162', '009164', '009165', '009171', '009182', '009189',
            '009213', '009214', '009218', '009219', '009221', '009226', '009227', '009235',
            '009242', '009263', '009273', '009284', '009298', '009299', '009310')

  """)

print(abx)

abx.to_csv('abx.csv',index=False)


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


       hadm_id  icustay_id  startdate_epoch  enddate_epoch
0       141008      295041       4961692800     4961865600
1       137321      213466       4986921600     4987353600
2       192751      275950       6788966400     6789052800
3       197487      239239       5460652800     5460739200
4       101039      267167       5115744000     5115830400
...        ...         ...              ...            ...
19732   107207        <NA>       5167152000     5167152000
19733   118558        <NA>       5825865600     5825952000
19734   116250        <NA>       6738854400     6738940800
19735   140100        <NA>       6692112000     6692284800
19736   154584        <NA>       5649177600     5649264000

[19737 rows x 4 columns]




4. demog (Patient demographics)
See https://github.com/MIT-LCP/mimic-code/blob/master/concepts/comorbidity/elixhauser-quan.sql
This code calculates the Elixhauser comorbidities as defined in Quan et. al 2009
This outputs a materialized view (table) with 58976 rows and 31 columns. The first column is 'hadm_id' and the
rest of the columns are as given below (Each entry is either 0 or 1):
2. 'congestive_heart_failure',
3. 'cardiac_arrhythmias',
4. 'valvular_disease',
5. 'pulmonary_circulation',
6. 'peripheral_vascular',
7. 'hypertension',
8. 'paralysis',
9. 'other_neurological'
10.'chronic_pulmonary',
11. 'diabetes_uncomplicated',
12. 'diabetes_complicated',
13. 'hypothyroidism',
14. 'renal_failure',
15. 'liver_disease',
16. 'peptic_ulcer',
17. 'aids',
18. 'lymphoma',
19. 'metastatic_cancer',
20. 'solid_tumor',
21. 'rheumatoid_arthritis',
22. 'coagulopathy',
23. 'obesity',
24. 'weight_loss',
25. 'fluid_electrolyte',
26. 'blood_loss_anemia',
27. 'deficiency_anemias',
28. 'alcohol_abuse',
29. 'drug_abuse',
30. 'psychoses',
31. 'depression'

In [ ]:
demography = run_query("""
WITH eliflg AS (
  -- Create flags for each condition based on the ICD-9 codes
  SELECT hadm_id,
         seq_num,
         icd9_code,
         CASE
           WHEN icd9_code IN ('39891','40201','40211','40291','40401','40403','40411','40413','40491','40493') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('4254','4255','4257','4258','4259') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('428') THEN 1
           ELSE 0
         END AS chf, -- Congestive heart failure
         CASE
           WHEN icd9_code IN ('42613','42610','42612','99601','99604') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('4260','4267','4269','4270','4271','4272','4273','4274','4276','4278','4279','7850','V450','V533') THEN 1
           ELSE 0
         END AS arrhy, -- Arrhythmia
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('0932','7463','7464','7465','7466','V422','V433') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('394','395','396','397','424') THEN 1
           ELSE 0
         END AS valve, -- Valvular disease
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('4150','4151','4170','4178','4179') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('416') THEN 1
           ELSE 0
         END AS pulmcirc, -- Pulmonary circulation disorder
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('0930','4373','4431','4432','4438','4439','4471','5571','5579','V434') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('440','441') THEN 1
           ELSE 0
         END AS perivasc, -- Peripheral vascular disease
         CASE
           WHEN SUBSTR(icd9_code, 1, 3) IN ('401') THEN 1
           ELSE 0
         END AS htn, -- Hypertension, uncomplicated
         CASE
           WHEN SUBSTR(icd9_code, 1, 3) IN ('402','403','404','405') THEN 1
           ELSE 0
         END AS htncx, -- Hypertension, complicated
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('3341','3440','3441','3442','3443','3444','3445','3446','3449') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('342','343') THEN 1
           ELSE 0
         END AS para, -- Paralysis
         CASE
           WHEN icd9_code IN ('33392') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('3319','3320','3321','3334','3335','3362','3481','3483','7803','7843') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('334','335','340','341','345') THEN 1
           ELSE 0
         END AS neuro, -- Other neurological
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('4168','4169','5064','5081','5088') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('490','491','492','493','494','495','496','500','501','502','503','504','505') THEN 1
           ELSE 0
         END AS chrnlung, -- Chronic pulmonary disease
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2500','2501','2502','2503') THEN 1
           ELSE 0
         END AS dm, -- Diabetes w/o chronic complications
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2504','2505','2506','2507','2508','2509') THEN 1
           ELSE 0
         END AS dmcx, -- Diabetes w/ chronic complications
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2409','2461','2468') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('243','244') THEN 1
           ELSE 0
         END AS hypothy, -- Hypothyroidism
         CASE
           WHEN icd9_code IN ('40301','40311','40391','40402','40403','40412','40413','40492','40493') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('5880','V420','V451') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('585','586','V56') THEN 1
           ELSE 0
         END AS renlfail, -- Renal failure
         CASE
           WHEN icd9_code IN ('07022','07023','07032','07033','07044','07054') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('0706','0709','4560','4561','4562','5722','5723','5724','5728','5733','5734','5738','5739','V427') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('570','571') THEN 1
           ELSE 0
         END AS liver, -- Liver disease
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('5317','5319','5327','5329','5337','5339','5347','5349') THEN 1
           ELSE 0
         END AS ulcer, -- Chronic Peptic ulcer disease
         CASE
           WHEN SUBSTR(icd9_code, 1, 3) IN ('042','043','044') THEN 1
           ELSE 0
         END AS aids, -- HIV and AIDS
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2030','2386') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('200','201','202') THEN 1
           ELSE 0
         END AS lymph, -- Lymphoma
         CASE
           WHEN SUBSTR(icd9_code, 1, 3) IN ('196','197','198','199') THEN 1
           ELSE 0
         END AS mets, -- Metastatic cancer
         CASE
           WHEN SUBSTR(icd9_code, 1, 3) IN ('140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156','157','158','159','160','161','162','163','164','165','166','167','168','169','170','171','172','174','175','176','177','178','179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194','195') THEN 1
           ELSE 0
         END AS tumor, -- Solid tumor without metastasis
         CASE
           WHEN icd9_code IN ('72889','72930') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('7010','7100','7101','7102','7103','7104','7108','7109','7112','7193','7285') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('446','714','720','725') THEN 1
           ELSE 0
         END AS arth, -- Rheumatoid arthritis/collagen vascular diseases
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2871','2873','2874','2875') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('286') THEN 1
           ELSE 0
         END AS coag, -- Coagulation deficiency
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2780') THEN 1
           ELSE 0
         END AS obese, -- Obesity
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('7832','7994') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('260','261','262','263') THEN 1
           ELSE 0
         END AS wghtloss, -- Weight loss
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2536') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('276') THEN 1
           ELSE 0
         END AS lytes, -- Fluid and electrolyte disorders
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2800') THEN 1
           ELSE 0
         END AS bldloss, -- Blood loss anemia
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2801','2808','2809') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('281') THEN 1
           ELSE 0
         END AS anemdef, -- Deficiency anemias
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2652','2911','2912','2913','2915','2918','2919','3030','3039','3050','3575','4255','5353','5710','5711','5712','5713','V113') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('980') THEN 1
           ELSE 0
         END AS alcohol, -- Alcohol abuse
         CASE
           WHEN icd9_code IN ('V6542') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('3052','3053','3054','3055','3056','3057','3058','3059') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('292','304') THEN 1
           ELSE 0
         END AS drug, -- Drug abuse
         CASE
           WHEN icd9_code IN ('29604','29614','29644','29654') THEN 1
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2938') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('295','297','298') THEN 1
           ELSE 0
         END AS psych, -- Psychoses
         CASE
           WHEN SUBSTR(icd9_code, 1, 4) IN ('2962','2963','2965','3004') THEN 1
           WHEN SUBSTR(icd9_code, 1, 3) IN ('309','311') THEN 1
           ELSE 0
         END AS depress -- Depression
  FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
  WHERE seq_num != 1 -- Exclude primary diagnosis
)
-- Now we aggregate the flags to summarize comorbidities per patient admission
, eligrp AS (
  SELECT
    hadm_id,
    MAX(chf) AS chf,             -- Congestive heart failure
    MAX(arrhy) AS arrhy,         -- Arrhythmias
    MAX(valve) AS valve,         -- Valvular disease
    MAX(pulmcirc) AS pulmcirc,   -- Pulmonary circulation disorders
    MAX(perivasc) AS perivasc,   -- Peripheral vascular disorders
    MAX(htn) AS htn,             -- Hypertension (uncomplicated)
    MAX(htncx) AS htncx,         -- Hypertension (complicated)
    MAX(para) AS para,           -- Paralysis
    MAX(neuro) AS neuro,         -- Other neurological disorders
    MAX(chrnlung) AS chrnlung,   -- Chronic pulmonary disease
    MAX(dm) AS dm,               -- Diabetes w/o chronic complications
    MAX(dmcx) AS dmcx,           -- Diabetes w/ chronic complications
    MAX(hypothy) AS hypothy,     -- Hypothyroidism
    MAX(renlfail) AS renlfail,   -- Renal failure
    MAX(liver) AS liver,         -- Liver disease
    MAX(ulcer) AS ulcer,         -- Chronic peptic ulcer disease
    MAX(aids) AS aids,           -- HIV/AIDS
    MAX(lymph) AS lymph,         -- Lymphoma
    MAX(mets) AS mets,           -- Metastatic cancer
    MAX(tumor) AS tumor,         -- Solid tumor without metastasis
    MAX(arth) AS arth,           -- Rheumatoid arthritis/collagen vascular diseases
    MAX(coag) AS coag,           -- Coagulation deficiency
    MAX(obese) AS obese,         -- Obesity
    MAX(wghtloss) AS wghtloss,   -- Weight loss
    MAX(lytes) AS lytes,         -- Fluid and electrolyte disorders
    MAX(bldloss) AS bldloss,     -- Blood loss anemia
    MAX(anemdef) AS anemdef,     -- Deficiency anemias
    MAX(alcohol) AS alcohol,     -- Alcohol abuse
    MAX(drug) AS drug,           -- Drug abuse
    MAX(psych) AS psych,         -- Psychoses
    MAX(depress) AS depress      -- Depression
  FROM eliflg
  GROUP BY hadm_id
)

-- Final query to join the aggregated comorbidity flags with the main admissions data


SELECT
    ad.subject_id,
    ad.hadm_id,
    i.icustay_id,
    UNIX_SECONDS(CAST(ad.admittime AS TIMESTAMP)) AS admittime_epoch,
    UNIX_SECONDS(CAST(ad.dischtime AS TIMESTAMP)) AS dischtime_epoch,
    ROW_NUMBER() OVER (PARTITION BY ad.subject_id ORDER BY i.intime ASC) AS adm_order,


    CASE WHEN i.first_careunit = 'NICU' THEN 5
         WHEN i.first_careunit = 'SICU' THEN 2
         WHEN i.first_careunit = 'CSRU' THEN 4
         WHEN i.first_careunit = 'CCU' THEN 6
         WHEN i.first_careunit = 'MICU' THEN 1
         WHEN i.first_careunit = 'TSICU' THEN 3
         ELSE NULL END AS unit,
    UNIX_SECONDS(CAST(i.intime AS TIMESTAMP)) AS intime_epoch,
    UNIX_SECONDS(CAST(i.outtime AS TIMESTAMP)) AS outtime_epoch,
    i.los,
    TIMESTAMP_DIFF(CAST(i.intime AS TIMESTAMP), CAST(p.dob AS TIMESTAMP), DAY) AS age_days,
    UNIX_SECONDS(CAST(p.dob AS TIMESTAMP)) AS dob_epoch,
    UNIX_SECONDS(CAST(p.dod AS TIMESTAMP)) AS dod_epoch,
    p.expire_flag,
    CASE WHEN p.gender = 'M' THEN 1 WHEN p.gender = 'F' THEN 2 ELSE NULL END AS gender,
    -- Mortality checks
   -- Mortality calculations: Using TIMESTAMP_DIFF instead of age
    CAST(TIMESTAMP_DIFF(CAST(p.dod AS TIMESTAMP), CAST(ad.dischtime AS TIMESTAMP), SECOND) <= 24 * 3600 AS INT) AS morta_hosp,
    CAST(TIMESTAMP_DIFF(CAST(p.dod AS TIMESTAMP), CAST(i.intime AS TIMESTAMP), SECOND) <= 90 * 24 * 3600 AS INT) AS morta_90,
    -- Elixhauser comorbidity score: Sum the comorbidities
    eli.chf + eli.arrhy + eli.valve + eli.pulmcirc + eli.perivasc +
    CASE WHEN eli.htn = 1 OR eli.htncx = 1 THEN 1 ELSE 0 END AS hypertension,
    eli.para + eli.neuro + eli.chrnlung + eli.dm + eli.dmcx + eli.hypothy +
    eli.renlfail + eli.liver + eli.ulcer + eli.aids + eli.lymph + eli.mets + eli.tumor +
    eli.arth + eli.coag + eli.obese + eli.wghtloss + eli.lytes + eli.bldloss + eli.anemdef +
    eli.alcohol + eli.drug + eli.psych + eli.depress AS elixhauser

FROM `physionet-data.mimiciii_clinical.admissions` ad
JOIN `physionet-data.mimiciii_clinical.icustays` i ON ad.hadm_id = i.hadm_id
JOIN `physionet-data.mimiciii_clinical.patients` p ON p.subject_id = ad.subject_id
LEFT JOIN eligrp eli ON ad.hadm_id = eli.hadm_id

ORDER BY ad.subject_id ASC, i.intime ASC;




""")


print(demography)

demography.to_csv('demography.csv',index=False)



<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


       subject_id  hadm_id  icustay_id  admittime_epoch  dischtime_epoch  \
0               2   163353      243653       5318679840       5319013680   
1               3   145834      211552       4159278480       4160210280   
2               4   185777      294638       6980516880       6981187260   
3               5   178980      214757       4199833860       4200034500   
4               6   107064      228232       6482099700       6483513600   
...           ...      ...         ...              ...              ...   
61527       99985   176670      279638       6660845220       6662279100   
61528       99991   151118      226241       6784158600       6785208900   
61529       99992   197084      242052       5508756180       5509014960   
61530       99995   137810      229633       5588899200       5589177300   
61531       99999   113369      246512       4670291700       4670757000   

       adm_order  unit  intime_epoch  outtime_epoch      los  age_days  \
0            

In [ ]:
demography.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   subject_id       61532 non-null  Int64  
 1   hadm_id          61532 non-null  Int64  
 2   icustay_id       61532 non-null  Int64  
 3   admittime_epoch  61532 non-null  Int64  
 4   dischtime_epoch  61532 non-null  Int64  
 5   adm_order        61532 non-null  Int64  
 6   unit             61532 non-null  Int64  
 7   intime_epoch     61532 non-null  Int64  
 8   outtime_epoch    61522 non-null  Int64  
 9   los              61522 non-null  float64
 10  age_days         61532 non-null  Int64  
 11  dob_epoch        61532 non-null  Int64  
 12  dod_epoch        24191 non-null  Int64  
 13  expire_flag      61532 non-null  Int64  
 14  gender           61532 non-null  Int64  
 15  morta_hosp       24191 non-null  Int64  
 16  morta_90         24191 non-null  Int64  
 17  hypertension

In [ ]:

# 5. ce (Patient vitals from chartevents)
# Divided into 10 chunks for speed (indexed by ICU stay ID). Each chunk is around 170 MB.
# Each itemid here corresponds to single measurement type

for i in range(0,100000,10000):
  print(i)
  ce = run_query("""
  SELECT DISTINCT
    icustay_id,
    UNIX_SECONDS(CAST(charttime AS TIMESTAMP)) AS charttime,
    itemid,
    CASE
        WHEN value = 'None' THEN 0
        WHEN value = 'Ventilator' THEN 1
        WHEN value = 'Cannula' THEN 2
        WHEN value = 'Nasal Cannula' THEN 2
        WHEN value = 'Face Tent' THEN 3
        WHEN value = 'Aerosol-Cool' THEN 4
        WHEN value = 'Trach Mask' THEN 5
        WHEN value = 'Hi Flow Neb' THEN 6
        WHEN value = 'Non-Rebreather' THEN 7
        WHEN value = '' THEN 8
        WHEN value = 'Venti Mask' THEN 9
        WHEN value = 'Medium Conc Mask' THEN 10
        ELSE valuenum
    END AS valuenum
FROM `physionet-data.mimiciii_clinical.chartevents`
WHERE icustay_id >= 200000 AND icustay_id < 210000
  AND value IS NOT NULL
  AND itemid IN (467, 470, 471, 223834, 227287, 194, 224691, 226707, 226730, 581, 580,
                 224639, 226512, 198, 228096, 211, 220045, 220179, 225309, 6701, 6,
                 227243, 224167, 51, 455, 220181, 220052, 225312, 224322, 6702, 443,
                 52, 456, 8368, 8441, 225310, 8555, 8440, 220210, 3337, 224422, 618,
                 3603, 615, 220277, 646, 834, 3655, 223762, 223761, 678, 220074, 113,
                 492, 491, 8448, 116, 1372, 1366, 228368, 228177, 626, 223835, 3420,
                 160, 727, 190, 220339, 506, 505, 224700, 224686, 224684, 684, 224421,
                 224687, 450, 448, 445, 224697, 444, 224695, 535, 224696, 543, 3083,
                 2566, 654, 3050, 681, 2311)
ORDER BY icustay_id, charttime;

  """ )

  print(ce)
  ce.to_csv('ce' + str(i)+str(i+10000) +'.csv',index=False,sep='|')

0


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


         icustay_id   charttime  itemid  valuenum
0            200001  6686996760  220045     115.0
1            200001  6686996820  220210      22.0
2            200001  6686996880  220181      77.0
3            200001  6686996880  220179     113.0
4            200001  6686997240  220277      94.0
...             ...         ...     ...       ...
5295916      209999  5191568100     646      96.0
5295917      209999  5191568100    8441      44.0
5295918      209999  5191568100     618      34.0
5295919      209999  5191568100     456      64.0
5295920      209999  5191568100     211      89.0

[5295921 rows x 4 columns]
10000


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


         icustay_id   charttime  itemid  valuenum
0            200001  6686996760  220045     115.0
1            200001  6686996820  220210      22.0
2            200001  6686996880  220181      77.0
3            200001  6686996880  220179     113.0
4            200001  6686997240  220277      94.0
...             ...         ...     ...       ...
5295916      209999  5191568100     646      96.0
5295917      209999  5191568100    8441      44.0
5295918      209999  5191568100     618      34.0
5295919      209999  5191568100     456      64.0
5295920      209999  5191568100     211      89.0

[5295921 rows x 4 columns]
20000


...

         icustay_id   charttime  itemid  valuenum
0            200001  6686996760  220045     115.0
1            200001  6686996820  220210      22.0
2            200001  6686996880  220181      77.0
3            200001  6686996880  220179     113.0
4            200001  6686997240  220277      94.0
...             ...         ...     ...       ...
5295916      209999  5191568100     646      96.0
5295917      209999  5191568100    8441      44.0
5295918      209999  5191568100     618      34.0
5295919      209999  5191568100     456      64.0
5295920      209999  5191568100     211      89.0

[5295921 rows x 4 columns]
80000


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


         icustay_id   charttime  itemid  valuenum
0            200001  6686996760  220045     115.0
1            200001  6686996820  220210      22.0
2            200001  6686996880  220181      77.0
3            200001  6686996880  220179     113.0
4            200001  6686997240  220277      94.0
...             ...         ...     ...       ...
5295916      209999  5191568100     646      96.0
5295917      209999  5191568100    8441      44.0
5295918      209999  5191568100     618      34.0
5295919      209999  5191568100     456      64.0
5295920      209999  5191568100     211      89.0

[5295921 rows x 4 columns]
90000


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


         icustay_id   charttime  itemid  valuenum
0            200001  6686996760  220045     115.0
1            200001  6686996820  220210      22.0
2            200001  6686996880  220181      77.0
3            200001  6686996880  220179     113.0
4            200001  6686997240  220277      94.0
...             ...         ...     ...       ...
5295916      209999  5191568100     646      96.0
5295917      209999  5191568100    8441      44.0
5295918      209999  5191568100     618      34.0
5295919      209999  5191568100     456      64.0
5295920      209999  5191568100     211      89.0

[5295921 rows x 4 columns]


In [9]:

# 6. labs_ce (Labs from chartevents)
# Each itemid here corresponds to single measurement type
labs_ce = run_query("""
SELECT
    icustay_id,
    UNIX_SECONDS(CAST(charttime AS TIMESTAMP)) AS charttime,
    itemid,
    valuenum
FROM
    `physionet-data.mimiciii_clinical.chartevents`
WHERE
    valuenum IS NOT NULL
    AND icustay_id IS NOT NULL
    AND itemid IN
    (829, 1535, 227442, 227464, 4195, 3726, 3792, 837, 220645, 4194,
    3725, 3803, 226534, 1536, 4195, 3726, 788, 220602, 1523, 4193, 3724,
    226536, 3747, 225664, 807, 811, 1529, 220621, 226537, 3744, 781, 1162, 225624,
    3737, 791, 1525, 220615, 3750, 821, 1532, 220635, 786, 225625, 1522, 3746, 816, 225667,
    3766, 777, 787, 770, 3801, 769, 3802, 1538, 848, 225690, 803, 1527, 225651, 3807,
    1539, 849, 772, 1521, 227456, 3727, 227429, 851, 227444, 814, 220228, 813,
    220545, 3761, 226540, 4197, 3799, 1127, 1542, 220546, 4200, 3834, 828, 227457,
    3789, 825, 1533, 227466, 3796, 824, 1286, 1671, 1520, 768, 220507, 815, 1530, 227467, 780,
    1126, 3839, 4753, 779, 490, 3785, 3838, 3837, 778, 3784, 3836, 3835, 776, 224828, 3736,
    4196, 3740, 74, 225668, 1531, 227443, 1817, 228640, 823, 227686)
order by icustay_id, charttime, itemid

""")

print(labs_ce)
labs_ce.to_csv('labs_ce.csv',index=False,sep='|')

<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


          icustay_id   charttime  itemid  valuenum
0             200001  6686998020  224828      1.00
1             200001  6686998020  225667      1.13
2             200001  6686998020  225668      1.40
3             200001  6686998020  227464      4.60
4             200001  6687007200  225664    118.00
...              ...         ...     ...       ...
10895291      299999  4659924840     829      4.50
10895292      299999  4659924840    1529    103.00
10895293      299999  4659924840    1535      4.50
10895294      299999  4659930000     807    117.00
10895295      299999  4659944400     807    130.00

[10895296 rows x 4 columns]
          icustay_id   charttime  itemid  valuenum
0             200001  6686998020  224828      1.00
1             200001  6686998020  225667      1.13
2             200001  6686998020  225668      1.40
3             200001  6686998020  227464      4.60
4             200001  6687007200  225664    118.00
...              ...         ...     ...       ...
10

In [10]:

# 7. labs_le (Labs from lab events)
labs_le = run_query("""
SELECT
    xx.icustay_id,
    UNIX_SECONDS(CAST(f.charttime AS TIMESTAMP)) AS timestp,
    f.itemid,
    f.valuenum
FROM (
    SELECT
        subject_id,
        hadm_id,
        icustay_id,
        intime,
        outtime
    FROM
        `physionet-data.mimiciii_clinical.icustays`
) AS xx
INNER JOIN
    `physionet-data.mimiciii_clinical.labevents` AS f
    ON f.hadm_id = xx.hadm_id
    AND f.charttime BETWEEN TIMESTAMP_SUB(xx.intime, INTERVAL 1 DAY)
                        AND TIMESTAMP_ADD(xx.outtime, INTERVAL 1 DAY)
    AND f.itemid IN (
        50971, 50822, 50824, 50806, 50931, 51081, 50885, 51003, 51222, 50810,
        51301, 50983, 50902, 50809, 51006, 50912, 50960, 50893, 50808, 50804,
        50878, 50861, 51464, 50883, 50976, 50862, 51002, 50889, 50811, 51221,
        51279, 51300, 51265, 51275, 51274, 51237, 50820, 50821, 50818, 50802,
        50813, 50882, 50803
    )
    AND f.valuenum IS NOT NULL
ORDER BY
    f.hadm_id, timestp, f.itemid;

""")

print (labs_le)
labs_le.to_csv('labs_le.csv',index=False,sep='|')


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(
<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


          icustay_id     timestp  itemid  valuenum
0             275225  4660791720   50861     10.00
1             275225  4660791720   50878     16.00
2             275225  4660791720   50882     11.00
3             275225  4660791720   50885      0.50
4             275225  4660791720   50902    101.00
...              ...         ...     ...       ...
10612606      211153  5247077280   51221     37.90
10612607      211153  5247077280   51222     12.60
10612608      211153  5247077280   51265    539.00
10612609      211153  5247077280   51279      4.13
10612610      211153  5247077280   51301     10.30

[10612611 rows x 4 columns]
          icustay_id     timestp  itemid  valuenum
0             275225  4660791720   50861     10.00
1             275225  4660791720   50878     16.00
2             275225  4660791720   50882     11.00
3             275225  4660791720   50885      0.50
4             275225  4660791720   50902    101.00
...              ...         ...     ...       ...
10

In [ ]:

# 8. uo (Real-time Urine Output)
uo = run_query("""
SELECT
    icustay_id,
    UNIX_SECONDS(CAST(charttime AS TIMESTAMP)) AS charttime,
    itemid,
    value
FROM
    `physionet-data.mimiciii_clinical.outputevents`
WHERE
    icustay_id IS NOT NULL
    AND value IS NOT NULL
    AND itemid IN (
        40055, 43175, 40069, 40094, 40715, 40473, 40085, 40057, 40056, 40405,
        40428, 40096, 40651, 226559, 226560, 227510, 226561, 227489, 226584,
        226563, 226564, 226565, 226557, 226558
    )
ORDER BY
    icustay_id, charttime, itemid;

""")

print(uo)
uo.to_csv('uo.csv',index=False,sep='|')


<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(
<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [ ]:
# 9. preadm_uo (Pre-admission Urine Output)
preadm_uo = run_query("""
SELECT DISTINCT
    oe.icustay_id,
    UNIX_SECONDS(CAST(oe.charttime AS TIMESTAMP)) AS charttime,
    oe.itemid,
    oe.value,
    -- Calculate difference in minutes between ic.intime and oe.charttime
    TIMESTAMP_DIFF(ic.intime, oe.charttime, MINUTE) AS datediff_minutes
FROM
    `physionet-data.mimiciii_clinical.outputevents` oe
JOIN
    `physionet-data.mimiciii_clinical.icustays` ic
ON
    oe.icustay_id = ic.icustay_id
WHERE
    oe.itemid IN (40060, 226633)
ORDER BY
    icustay_id, charttime, itemid;

""")

print(preadm_uo)
preadm_uo.to_csv('preadm_uo.csv',index=False,sep='|')



In [ ]:

# 10. fluid_mv (Real-time input from metavision)
# This extraction converts the different rates and dimensions to a common unit
"""
Records with no rate = STAT
Records with rate = INFUSION
fluids corrected for tonicity
"""
fluid_mv = run_query("""
with t1 as
(
select icustay_id, UNIX_SECONDS(CAST(starttime AS TIMESTAMP)) AS starttime, UNIX_SECONDS(CAST(endtime AS TIMESTAMP)) AS endtime, itemid, amount, rate,
case when itemid in (30176, 30315) then amount *0.25
when itemid in (30161) then amount *0.3
when itemid in (30020, 30015, 225823, 30321, 30186, 30211, 30353, 42742, 42244, 225159) then amount *0.5 --
when itemid in (227531) then amount *2.75
when itemid in (30143, 225161) then amount *3
when itemid in (30009, 220862) then amount *5
when itemid in (30030, 220995, 227533) then amount *6.66
when itemid in (228341) then amount *8
else amount end as tev -- total equivalent volume
from `physionet-data.mimiciii_clinical.inputevents_mv`
-- only real time items !!
where icustay_id is not null and amount is not null and itemid in (225158, 225943, 226089, 225168,
225828, 220862, 220970, 220864, 225159, 220995, 225170, 225825, 227533, 225161, 227531, 225171, 225827,
225941, 225823, 228341, 30018, 30021, 30015, 30296, 30020, 30066, 30001, 30030,
30060, 30005, 30321, 30006, 30061, 30009, 30179, 30190, 30143, 30160, 30008, 30168, 30186, 30211, 30353, 30159, 30007,
30185, 30063, 30094, 30352, 30014, 30011, 30210, 46493, 45399, 46516, 40850, 30176, 30161, 30381, 30315, 42742, 30180,
46087, 41491, 30004, 42698, 42244)
)
select icustay_id, starttime, endtime, itemid, round(cast(amount as numeric),3) as amount,
round(cast(rate as numeric),3) as rate,round(cast(tev as numeric),3) as tev -- total equiv volume
from t1
order by icustay_id, starttime, itemid

""")

fluid_mv.to_csv('fluid_mv.csv',index=False,sep='|')

In [ ]:

# 11. fluid_cv (Real-time input from metavision)
# This extraction converts the different rates and dimensions to a common unit
"""
Records with no rate = STAT
Records with rate = INFUSION
fluids corrected for tonicity
"""
fluid_cv = run_query("""
WITH t1 AS (
    SELECT
        icustay_id,
        UNIX_SECONDS(CAST(CHARTTIME AS TIMESTAMP)) AS charttime,
        UNIX_SECONDS(CAST(STORETIME AS TIMESTAMP)) AS storetime,
        itemid,
        amount,
        rate,
        CASE
            WHEN itemid IN (30176, 30315) THEN amount * 0.25
            WHEN itemid IN (30161) THEN amount * 0.3
            WHEN itemid IN (30020, 30015, 225823, 30321, 30186, 30211, 30353, 42742, 42244, 225159) THEN amount * 0.5
            WHEN itemid IN (227531) THEN amount * 2.75
            WHEN itemid IN (30143, 225161) THEN amount * 3
            WHEN itemid IN (30009, 220862) THEN amount * 5
            WHEN itemid IN (30030, 220995, 227533) THEN amount * 6.66
            WHEN itemid IN (228341) THEN amount * 8
            ELSE amount
        END AS tev  -- total equivalent volume
    FROM
        `physionet-data.mimiciii_clinical.inputevents_cv`
    WHERE
        icustay_id IS NOT NULL
        AND amount IS NOT NULL
        AND itemid IN (
            225158, 225943, 226089, 225168, 225828, 220862, 220970, 220864, 225159,
            220995, 225170, 225825, 227533, 225161, 227531, 225171, 225827, 225941,
            225823, 228341, 30018, 30021, 30015, 30296, 30020, 30066, 30001, 30030,
            30060, 30005, 30321, 30006, 30061, 30009, 30179, 30190, 30143, 30160,
            30008, 30168, 30186, 30211, 30353, 30159, 30007, 30185, 30063, 30094,
            30352, 30014, 30011, 30210, 46493, 45399, 46516, 40850, 30176, 30161,
            30381, 30315, 42742, 30180, 46087, 41491, 30004, 42698, 42244
        )
)
SELECT
    icustay_id,
    charttime,
    storetime,
    itemid,
    ROUND(CAST(amount AS NUMERIC), 3) AS amount,
    ROUND(CAST(rate AS NUMERIC), 3) AS rate,
    ROUND(CAST(tev AS NUMERIC), 3) AS tev  -- total equivalent volume
FROM
    t1
ORDER BY
    icustay_id, charttime, itemid;


""")

fluid_cv.to_csv('fluid_cv.csv',index=False,sep='|')

In [ ]:

# 12. preadm_fluid (Pre-admission fluid intake)
preadm_fluid = run_query("""
with mv as
(
select ie.icustay_id, sum(ie.amount) as sum
from `physionet-data.mimiciii_clinical.inputevents_mv` ie, `physionet-data.mimiciii_clinical.d_items` ci
where ie.itemid=ci.itemid and ie.itemid in (30054, 30055, 30101, 30102, 30103, 30104, 30105, 30108, 226361,
226363, 226364, 226365, 226367, 226368, 226369, 226370, 226371, 226372, 226375, 226376, 227070, 227071, 227072)
group by icustay_id
), cv as
(
select ie.icustay_id, sum(ie.amount) as sum
from `physionet-data.mimiciii_clinical.inputevents_cv` ie, `physionet-data.mimiciii_clinical.d_items` ci
where ie.itemid=ci.itemid and ie.itemid in (30054, 30055, 30101, 30102, 30103, 30104, 30105, 30108, 226361,
226363, 226364, 226365, 226367, 226368, 226369, 226370, 226371, 226372, 226375, 226376, 227070, 227071, 227072)
group by icustay_id
)

select pt.icustay_id,
case when mv.sum is not null then mv.sum
when cv.sum is not null then cv.sum
else null end as inputpreadm
from `physionet-data.mimiciii_clinical.icustays` pt
left outer join mv
on mv.icustay_id=pt.icustay_id
left outer join cv
on cv.icustay_id=pt.icustay_id
order by icustay_id
""")

preadm_fluid.to_csv('preadm_fluid.csv',index=False,sep='|')


In [ ]:
# 13. vaso_mv (Vasopressors from metavision)
# This extraction converts the different rates and dimensions to a common units
"""
Drugs converted in noradrenaline-equivalent
Body weight assumed 80 kg when missing
"""
vaso_mv = run_query("""
SELECT
    icustay_id,
    itemid,
    UNIX_SECONDS(CAST(starttime AS TIMESTAMP)) AS starttime,
    UNIX_SECONDS(CAST(endtime AS TIMESTAMP)) AS endtime,
    -- Standardized rate calculation
    CASE
        WHEN itemid IN (30120, 221906, 30047) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate AS NUMERIC), 3)  -- norad
        WHEN itemid IN (30120, 221906, 30047) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate / 80 AS NUMERIC), 3)  -- norad
        WHEN itemid IN (30119, 221289) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate AS NUMERIC), 3)  -- epi
        WHEN itemid IN (30119, 221289) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate / 80 AS NUMERIC), 3)  -- epi
        WHEN itemid IN (30051, 222315) AND rate > 0.2 THEN ROUND(CAST(rate * 5 / 60 AS NUMERIC), 3)  -- vasopressin, in U/h
        WHEN itemid IN (30051, 222315) AND rateuom = 'units/min' THEN ROUND(CAST(rate * 5 AS NUMERIC), 3)  -- vasopressin
        WHEN itemid IN (30051, 222315) AND rateuom = 'units/hour' THEN ROUND(CAST(rate * 5 / 60 AS NUMERIC), 3)  -- vasopressin
        WHEN itemid IN (30128, 221749, 30127) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate * 0.45 AS NUMERIC), 3)  -- phenyl
        WHEN itemid IN (30128, 221749, 30127) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate * 0.45 / 80 AS NUMERIC), 3)  -- phenyl
        WHEN itemid IN (221662, 30043, 30307) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate * 0.01 AS NUMERIC), 3)  -- dopa
        WHEN itemid IN (221662, 30043, 30307) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate * 0.01 / 80 AS NUMERIC), 3)  -- dopa
        ELSE NULL
    END AS rate_std  -- Standardized rate
FROM
    `physionet-data.mimiciii_clinical.inputevents_mv`
WHERE
    itemid IN (30128, 30120, 30051, 221749, 221906, 30119, 30047,
               30127, 221289, 222315, 221662, 30043, 30307)
    AND rate IS NOT NULL
    AND statusdescription <> 'Rewritten'
ORDER BY
    icustay_id, itemid, starttime;

""")

vaso_mv.to_csv('vaso_mv.csv',index=False,sep='|')

In [ ]:

# 14. vaso_cv (Vasopressors from carevue)
# This extraction converts the different rates and dimensions to a common units

vaso_cv = run_query("""
SELECT
    icustay_id,
    itemid,
    UNIX_SECONDS(CAST(charttime AS TIMESTAMP)) AS charttime,
    UNIX_SECONDS(CAST(storetime AS TIMESTAMP)) AS storetime,
    -- Standardized rate calculation
    CASE
        WHEN itemid IN (30120, 221906, 30047) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate AS NUMERIC), 3)  -- norad
        WHEN itemid IN (30120, 221906, 30047) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate / 80 AS NUMERIC), 3)  -- norad
        WHEN itemid IN (30119, 221289) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate AS NUMERIC), 3)  -- epi
        WHEN itemid IN (30119, 221289) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate / 80 AS NUMERIC), 3)  -- epi
        WHEN itemid IN (30051, 222315) AND rate > 0.2 THEN ROUND(CAST(rate * 5 / 60 AS NUMERIC), 3)  -- vasopressin, in U/h
        WHEN itemid IN (30051, 222315) AND rateuom = 'units/min' THEN ROUND(CAST(rate * 5 AS NUMERIC), 3)  -- vasopressin
        WHEN itemid IN (30051, 222315) AND rateuom = 'units/hour' THEN ROUND(CAST(rate * 5 / 60 AS NUMERIC), 3)  -- vasopressin
        WHEN itemid IN (30128, 221749, 30127) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate * 0.45 AS NUMERIC), 3)  -- phenyl
        WHEN itemid IN (30128, 221749, 30127) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate * 0.45 / 80 AS NUMERIC), 3)  -- phenyl
        WHEN itemid IN (221662, 30043, 30307) AND rateuom = 'mcg/kg/min' THEN ROUND(CAST(rate * 0.01 AS NUMERIC), 3)  -- dopa
        WHEN itemid IN (221662, 30043, 30307) AND rateuom = 'mcg/min' THEN ROUND(CAST(rate * 0.01 / 80 AS NUMERIC), 3)  -- dopa
        ELSE NULL
    END AS rate_std  -- Standardized rate
FROM
    `physionet-data.mimiciii_clinical.inputevents_cv`
WHERE
    itemid IN (30128, 30120, 30051, 221749, 221906, 30119, 30047,
               30127, 221289, 222315, 221662, 30043, 30307)
    AND rate IS NOT NULL
ORDER BY
    icustay_id, itemid, charttime;

""")

vaso_cv.to_csv('vaso_cv.csv',index=False,sep='|')

In [18]:

# 15. mechvent (Mechanical ventilation)
mechvent = run_query("""
SELECT
    icustay_id,
    UNIX_SECONDS(CAST(charttime AS TIMESTAMP)) AS charttime,
    -- Determine whether it's an instance of mechanical ventilation
    MAX(
      CASE
        WHEN itemid IS NULL OR value IS NULL THEN 0 -- can't have null values
        WHEN itemid = 720 AND value != 'Other/Remarks' THEN 1 -- VentTypeRecorded
        WHEN itemid = 467 AND value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
        WHEN itemid IN (
          445, 448, 449, 450, 1340, 1486, 1600, 224687, -- minute volume
          639, 654, 681, 682, 683, 684, 224685, 224684, 224686, -- tidal volume
          218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 224747, -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
          221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 224750, 227187, -- Insp pressure
          543, -- PlateauPressure
          5865, 5866, 224707, 224709, 224705, 224706, -- APRV pressure
          60, 437, 505, 506, 686, 220339, 224700, -- PEEP
          3459, -- high pressure relief
          501, 502, 503, 224702, -- PCV
          223, 667, 668, 669, 670, 671, 672, -- TCPCV
          157, 158, 1852, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 8382, 227809, 227810, -- ETT
          224701 -- PSVlevel
        ) THEN 1
        ELSE 0
      END
    ) AS mechvent,

    -- Extubation determination
    MAX(
      CASE
        WHEN itemid IS NULL OR value IS NULL THEN 0
        WHEN itemid = 640 AND value IN ('Extubated', 'Self Extubation') THEN 1
        ELSE 0
      END
    ) AS extubated,

    -- Self-extubation determination
    MAX(
      CASE
        WHEN itemid IS NULL OR value IS NULL THEN 0
        WHEN itemid = 640 AND value = 'Self Extubation' THEN 1
        ELSE 0
      END
    ) AS selfextubated

FROM
    `physionet-data.mimiciii_clinical.chartevents` ce
WHERE
    value IS NOT NULL
    AND itemid IN (
      640, -- extubated
      720, -- vent type
      467, -- O2 delivery device
      445, 448, 449, 450, 1340, 1486, 1600, 224687, -- minute volume
      639, 654, 681, 682, 683, 684, 224685, 224684, 224686, -- tidal volume
      218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 224747, -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
      221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 224750, 227187, -- Insp pressure
      543, -- PlateauPressure
      5865, 5866, 224707, 224709, 224705, 224706, -- APRV pressure
      60, 437, 505, 506, 686, 220339, 224700, -- PEEP
      3459, -- high pressure relief
      501, 502, 503, 224702, -- PCV
      223, 667, 668, 669, 670, 671, 672, -- TCPCV
      157, 158, 1852, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 8382, 227809, 227810, -- ETT
      224701 -- PSVlevel
    )
GROUP BY
    icustay_id, charttime
ORDER BY
    icustay_id, charttime;

""")

mechvent.to_csv('mechvent.csv',index=False,sep='|')

<ipython-input-2-76babdfb79b9>:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(
